In [1]:
from colmap_loader import read_extrinsics_text, read_intrinsics_text, qvec2rotmat, \
            read_extrinsics_binary, read_intrinsics_binary
import os
import numpy as np
import cv2
def get_k_w2c(datadir, cam_id, timestamp):
    cameras_extrinsic_file = os.path.join(datadir, str(timestamp), "psiftproject/sparse/0/images.bin")
    cameras_intrinsic_file = os.path.join(datadir, str(timestamp), "psiftproject/sparse/0/cameras.bin")
    cam_extrinsics = read_extrinsics_binary(cameras_extrinsic_file)
    cam_intrinsics = read_intrinsics_binary(cameras_intrinsic_file)
    value = f'{cam_id}.png'
    for idx, key in enumerate(cam_extrinsics):
        if cam_extrinsics[key].name == value:
            extr_id = key
            intr_id = cam_extrinsics[key].camera_id

    extr = cam_extrinsics[extr_id]
    R = qvec2rotmat(extr.qvec)
    T = np.array(extr.tvec)
    params = cam_intrinsics[intr_id].params
    
    return params, R, T

def reproject_points(params, R, t, world_points):
    """
    使用SIMPLE_RADIAL相机模型将世界坐标系下的3D点投影到像素坐标 (优化版本，使用NumPy广播).

    参数：
        world_points (numpy.ndarray): 世界坐标系中的3D点，形状为(N, 3)
        params (list/numpy.ndarray): 相机内参及畸变参数 [焦距f, 主点cx, 主点cy, 畸变系数k]
        R (numpy.ndarray): 旋转矩阵，形状为(3, 3)
        t (numpy.ndarray): 平移向量，形状为(3,)

    返回：
        numpy.ndarray: 投影后的像素坐标，形状为 (N, 2)

    异常：
        ValueError: 当投影点的深度Z为0时抛出
    """
    # 解析相机参数
    f, cx, cy, k = params

    # 将世界坐标转换为相机坐标系
    P_cam = world_points @ R.T + t  # 形状(N, 3)
    X = P_cam[:, 0]
    Y = P_cam[:, 1]
    Z = P_cam[:, 2]

    # 检查深度有效性
    if np.any(np.isclose(Z, 0.0)):
        raise ValueError("投影失败：深度Z不能为零")

    # 归一化平面坐标
    u = X / Z
    v = Y / Z

    # 计算径向畸变
    r_sq = u**2 + v**2
    du = u * k * r_sq
    dv = v * k * r_sq

    # 应用畸变
    u_distorted = u + du
    v_distorted = v + dv

    # 转换为像素坐标
    x = f * u_distorted + cx
    y = f * v_distorted + cy

    points2d = np.stack([x, y], axis=-1)  # 形状 (N, 2)
    return points2d


def draw_points_on_image(points, image, color=(0, 0, 255), radius=5):
    """
    在图像上绘制二维点。

    Args:
        points: (m, 2) 的 NumPy 数组，包含要绘制的点的坐标。
        image_size: (height, width) 元组，指定图像的尺寸。
        color: (B, G, R) 元组，指定圆的颜色。 默认为红色 (0, 0, 255)。
        radius: 圆的半径。 默认为 5 像素。

    Returns:
        绘制了点的 NumPy 数组，表示图像。
    """

    # 创建一个空白图像（黑色背景）
 

    # 循环遍历点并绘制圆
    for point in points:
        # 确保点的坐标是整数
        x, y = map(int, point)

        # 确保点在图像范围内. 非常重要！
        if 0 <= x < 2200 and 0 <= y < 3216:
            cv2.circle(image, (x, y), radius, color, -1)  # -1 means filled circle
        else:
            print(f"Warning: Point ({x}, {y}) is out of image bounds.")

    return image

In [2]:
p3ds = [[
                2.8040075163175615,
                1.4869040415648964,
                6.6318295905343945
            ],
            [
                3.426516203156538,
                1.595201805452139,
                6.689766861826605
            ],
            [
                1.3418244698326989,
                1.3506115474673237,
                6.7099320450857505
            ],
            [
                1.42082493685548,
                1.3561597188432417,
                6.482682643291753
            ],
            [
                2.7867188232266855,
                1.5288614433787462,
                6.802562587631968
            ],
            [
                2.778233833437568,
                1.5018987842249558,
                6.560067832387631
            ],
            [
                3.346899275482407,
                1.5563960158065697,
                6.5275397086602425
            ],
            [
                3.2458821883493734,
                1.5306602300717511,
                6.7899247229142645
            ],
            [
                1.8755206573229941,
                1.3788777796804368,
                6.361524250585248
            ],
            [
                1.8658849519691305,
                1.3077491632817484,
                6.43913766019633
            ],
            [
                3.0673980273969743,
                1.5102755464678417,
                6.4352792824044975
            ],
            [
                1.9001178708634683,
                1.4188567307830766,
                6.4198005904672755
            ],
            [
                1.84054696199303,
                1.3840330299288581,
                6.352243484265676
            ],
            [
                1.374431292277464,
                1.3477230401594191,
                6.4626340349797715
            ],
            [
                3.3415448366710763,
                1.5313376180377434,
                6.409092403906946
            ],
            [
                1.9028887487447907,
                1.3225470313425554,
                6.381758054245322
            ],
            [
                2.6131878530802624,
                1.2855342143169821,
                5.3620846694830675
            ],
            [
                1.8678128350467773,
                1.3655116287679334,
                6.399510812115466
            ],
            [
                3.1116480336902583,
                1.508576649302455,
                6.483635094984109
            ],
            [
                2.1943795309649206,
                1.3931471581272463,
                6.509189994217842
            ],
            [
                3.229474135435934,
                1.4564631374989574,
                6.544060377033027
            ],
            [
                1.3430000857477573,
                1.32627039632683,
                6.6407422986847
            ],
            [
                0.0048904127097771566,
                0.37684561020749946,
                0.7192735533120113
            ],
            [
                2.07822776526271,
                1.4006312637015155,
                6.639781334699567
            ],
            [
                3.3595127084216685,
                1.4850454470965797,
                6.418484820644338
            ],
            [
                0.8642002144947085,
                0.8383249693744101,
                2.2432047529478467
            ],
            [
                0.276571479146984,
                0.4563040259201614,
                0.8088666593679328
            ],
            [
                2.792416255115773,
                1.5735638111290555,
                6.720664207414369
            ],
            [
                3.1508577317273776,
                1.5815640710187182,
                6.574933114320174
            ],
            [
                1.0897635623477093,
                0.7544444995057411,
                1.5140372087142282
            ],
            [
                0.0006095292689064884,
                0.3947029625940308,
                0.6614453968818284
            ],
            [
                3.1674864559148546,
                1.5866305710845126,
                6.465089684272954
            ],
            [
                1.352300844769033,
                1.347333083843317,
                6.624765743961561
            ],
            [
                3.4848001376819866,
                1.5711537226455619,
                6.665847258754155
            ],
            [
                3.362291300270769,
                1.552344539472685,
                6.592636055436401
            ],
            [
                1.4369919435440224,
                1.3895618205873248,
                6.513679074828033
            ],
            [
                3.738325501109415,
                1.4969914326634193,
                6.364903136415204
            ],
            [
                2.7590115809831595,
                1.298595816417338,
                5.60932089356568
            ],
            [
                1.3938440100096017,
                1.1933788066609026,
                5.764582412261092
            ],
            [
                1.9351542014904055,
                1.3986918285502519,
                6.524838500272406
            ],
            [
                2.5677774265971465,
                1.490280699837303,
                6.696874431523341
            ],
            [
                1.3970972261522583,
                1.3809701923876343,
                6.550906900562033
            ],
            [
                1.6447091322027596,
                1.3118336854482386,
                6.529511432081534
            ],
            [
                2.6068556447043267,
                1.4934996979629773,
                6.818810924551793
            ],
            [
                3.0932316945039613,
                1.5298648322472013,
                6.786748415906757
            ],
            [
                1.7813436850378355,
                1.373349707840306,
                6.666126615473626
            ],
            [
                3.013028359484876,
                1.5151268603297585,
                6.82223648393208
            ],
            [
                1.4075633568772075,
                1.436532024150602,
                6.689517285652581
            ],
            [
                3.3872986576396613,
                1.6060460813463209,
                6.708886072480809
            ],
            [
                0.00024997822084309,
                0.5497079908132001,
                0.5400054619286419
            ],
            [
                2.555631997053286,
                1.4824711480941302,
                6.755354853366301
            ],
            [
                2.5711774767215676,
                1.5152857795068915,
                6.792585772220889
            ],
            [
                2.9743783863948186,
                1.5239715540877699,
                6.760557444075221
            ],
            [
                2.5741134124928244,
                1.503107762121961,
                6.8408530332584405
            ],
            [
                1.4181485008272272,
                1.3953060707438163,
                6.691808833052457
            ],
            [
                1.8294353497651814,
                1.4076925723740072,
                6.897741552830093
            ],
            [
                2.654235171740895,
                1.5619992205615638,
                6.868372341859853
            ],
            [
                2.6265203467715157,
                1.481152085456383,
                6.789055713083208
            ],
            [
                1.2188973853381544,
                0.8838307632242026,
                6.718127881231396
            ],
            [
                2.6119978323045414,
                1.52637112019604,
                6.856844282043884
            ],
            [
                2.764526482828252,
                1.5289702326186543,
                6.917218835407937
            ],
            [
                2.547978280994096,
                1.4847949257473179,
                6.731294255512456
            ],
            [
                2.555388634245517,
                1.5004942400293544,
                6.76710559038554
            ],
            [
                2.8015128348611285,
                1.5399743545002877,
                6.8613713869822845
            ],
            [
                2.559741689649421,
                1.405936589466485,
                6.57503347063988
            ],
            [
                1.149113769640369,
                0.7317226518300011,
                6.985695710420167
            ],
            [
                2.6111752787926763,
                1.5004691821230935,
                6.880753456859149
            ],
            [
                1.3661622667350457,
                1.3573223434163424,
                6.638753157237289
            ],
            [
                2.6046739035400677,
                1.5160033584442019,
                6.825218303156852
            ],
            [
                2.9288069562373025,
                1.5266064729653905,
                6.882644164042986
            ],
            [
                2.5495748646206184,
                1.4492829203848547,
                6.805087518568985
            ],
            [
                2.6138654037567903,
                1.498625170238725,
                6.716098208674708
            ]]

In [2]:
p2ds = [[
                1550.5797119140625,
                1299.127685546875
            ],
            [
                1548.943359375,
                1303.380126953125
            ],
            [
                1547.2119140625,
                1301.2977294921875
            ],
            [
                1546.758056640625,
                1304.1385498046875
            ],
            [
                1547.2008056640625,
                1306.670654296875
            ],
            [
                1544.9970703125,
                1308.021484375
            ],
            [
                1544.2935791015625,
                1307.08056640625
            ],
            [
                1544.542236328125,
                1306.85205078125
            ],
            [
                1542.62451171875,
                1305.7353515625
            ],
            [
                1541.96630859375,
                1307.1539306640625
            ],
            [
                1539.55224609375,
                1308.1300048828125
            ],
            [
                1539.298828125,
                1307.9830322265625
            ],
            [
                1538.112548828125,
                1308.9212646484375
            ],
            [
                1537.3201904296875,
                1311.7347412109375
            ],
            [
                1537.1270751953125,
                1312.6416015625
            ],
            [
                1536.5875244140625,
                1313.0584716796875
            ],
            [
                1538.2451171875,
                1312.9417724609375
            ],
            [
                1538.1878662109375,
                1312.1676025390625
            ],
            [
                1538.1612548828125,
                1312.2906494140625
            ],
            [
                1541.1572265625,
                1311.9757080078125
            ],
            [
                1540.09814453125,
                1310.731201171875
            ],
            [
                1539.2864990234375,
                1309.8310546875
            ],
            [
                1538.63427734375,
                1307.0
            ],
            [
                1539.5390625,
                1302.682373046875
            ],
            [
                1539.7010498046875,
                1298.28125
            ],
            [
                1538.5706787109375,
                1294.34912109375
            ],
            [
                1536.2987060546875,
                1287.2001953125
            ],
            [
                1538.1307373046875,
                1279.073486328125
            ],
            [
                1537.8564453125,
                1273.322998046875
            ],
            [
                1536.94873046875,
                1268.4918212890625
            ],
            [
                1538.7540283203125,
                1260.5765380859375
            ],
            [
                1538.1103515625,
                1253.902099609375
            ],
            [
                1538.4219970703125,
                1248.81396484375
            ],
            [
                1538.6650390625,
                1244.381591796875
            ],
            [
                1538.5762939453125,
                1240.290771484375
            ],
            [
                1539.9759521484375,
                1235.8470458984375
            ],
            [
                1537.5352783203125,
                1232.9990234375
            ],
            [
                1535.3121337890625,
                1231.1263427734375
            ],
            [
                1535.790771484375,
                1229.728759765625
            ],
            [
                1533.1767578125,
                1228.2076416015625
            ],
            [
                1533.55712890625,
                1225.99169921875
            ],
            [
                1531.8514404296875,
                1224.8997802734375
            ],
            [
                1531.2337646484375,
                1222.9951171875
            ],
            [
                1526.739501953125,
                1223.1387939453125
            ],
            [
                1524.3759765625,
                1221.95166015625
            ],
            [
                1522.734375,
                1220.01318359375
            ],
            [
                1520.447509765625,
                1218.0382080078125
            ],
            [
                1520.6737060546875,
                1214.723388671875
            ],
            [
                1517.3438720703125,
                1214.1063232421875
            ],
            [
                1518.4139404296875,
                1212.74853515625
            ],
            [
                1516.7901611328125,
                1211.8857421875
            ],
            [
                1516.8402099609375,
                1210.04541015625
            ],
            [
                1516.607421875,
                1208.328857421875
            ],
            [
                1516.9716796875,
                1206.6700439453125
            ],
            [
                1514.1075439453125,
                1205.161865234375
            ],
            [
                1515.4482421875,
                1203.82080078125
            ],
            [
                1513.6617431640625,
                1203.236572265625
            ],
            [
                1512.423828125,
                1201.7579345703125
            ],
            [
                1510.496826171875,
                1201.0484619140625
            ],
            [
                1510.2371826171875,
                1200.2947998046875
            ],
            [
                1508.9962158203125,
                1199.2286376953125
            ],
            [
                1508.7921142578125,
                1198.3455810546875
            ],
            [
                1508.5706787109375,
                1198.4132080078125
            ],
            [
                1507.6639404296875,
                1198.1893310546875
            ],
            [
                1508.0692138671875,
                1195.8863525390625
            ],
            [
                1506.1431884765625,
                1195.85107421875
            ],
            [
                1505.57080078125,
                1195.5081787109375
            ],
            [
                1506.4464111328125,
                1193.5863037109375
            ],
            [
                1505.9910888671875,
                1193.12939453125
            ],
            [
                1503.944091796875,
                1192.6920166015625
            ],
            [
                1503.9720458984375,
                1191.78125
            ],
            [
                1504.9888916015625,
                1189.878662109375
            ]]

In [19]:
i = 49
datadir = f"/media/DGST_data/Test_Data/056/EMO-1-shout+laugh"
img_path = datadir+f"/{53+i}/psiftproject/images/9.png"
# point_cloud_path= f'/media/DGST_data/Test_Data//056/EMO-1-shout+laugh/1/psiftproject/dense/0/fused.ply' 

# import open3d as o3d
params, R, T = get_k_w2c(datadir, 9, i+53)

p2ds = reproject_points(params, R, T, [p3ds[i-1]])
print(p3ds[i-1])
print(p2ds)
a = np.any(p2ds < 0)
print(a)

img = cv2.imread(img_path)
draw_points_on_image(p2ds, img)
cv2.imwrite("test.png", img)
# print([p2ds[i-1]])



[3.3872986576396613, 1.6060460813463209, 6.708886072480809]
[[1517.61598234 1213.85246613]]
False


True

In [13]:
timestamp = 1
cam_id = 9
datadir = f"/media/DGST_data/Test_Data/056/EMO-1-shout+laugh"
cameras_extrinsic_file = os.path.join(datadir, str(timestamp), "psiftproject/sparse/0/images.bin")
cameras_intrinsic_file = os.path.join(datadir, str(timestamp), "psiftproject/sparse/0/cameras.bin")
cam_extrinsics = read_extrinsics_binary(cameras_extrinsic_file)
cam_intrinsics = read_intrinsics_binary(cameras_intrinsic_file)
value = f'{cam_id}.png'
for idx, key in enumerate(cam_extrinsics):
    if cam_extrinsics[key].name == value:
        extr_id = key
        intr_id = cam_extrinsics[key].camera_id

extr = cam_extrinsics[extr_id]
R = qvec2rotmat(extr.qvec)
T = np.array(extr.tvec)
params = cam_intrinsics[intr_id]

print(params)

Camera(id=16, model='SIMPLE_RADIAL', width=2200, height=3208, params=array([8.36925621e+03, 1.10000000e+03, 1.60400000e+03, 1.20526226e-01]))
